# Import model

In [1]:
model_path = 'C:/Users/Lenovo/Desktop/NLP/Final_project/test_run_model/fine_tuned_ModernBERT_Lora_512_b16_ver1_imdb'

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [2]:
import pandas as pd
df = pd.read_csv("C:/Users/Lenovo/Desktop/NLP/Final_project/unseen_review/review.csv")

df.head(10)

,sentiment,review
0,Negative,I had no background knowledge of this movie be...
1,Negative,I am a huge Jane Austen fan and I ordered the ...
2,Negative,Nothing to say but Wow! Has anyone actually ha...
3,Negative,i like Jane Austin novels. I love Pride and Pr...
4,Negative,In this day and age of incredible special movi...
5,Negative,Becky Harris plays the female shopper whose mi...
6,Negative,Watching TRUTH ABOUT LOVE (is this a double en...
7,Negative,"A painfully protracted, maudlin and predictabl..."
8,Negative,"Let's be honest. As a film school project, mad..."
9,Negative,A terrible movie as everyone has said. What ma...


In [3]:
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm  # progress bar

BATCH_SIZE = 8  # ✅ ปรับตาม VRAM

# ✅ เลือกอุปกรณ์ (GPU ถ้ามี, ถ้าไม่มีใช้ CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# ✅ ย้ายโมเดลไป GPU
model = model.to(device)

# ✅ โหลด CSV
df = pd.read_csv("C:/Users/Lenovo/Desktop/NLP/Final_project/unseen_review/review.csv")


all_pred_labels = []

# ✅ predict แบบ batch
for i in tqdm(range(0, len(df), BATCH_SIZE)):
    batch_texts = df["review"].iloc[i:i+BATCH_SIZE].tolist()
    
    # ✅ tokenize + ส่งไป GPU
    inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(device)
    
    # ✅ predict
    with torch.no_grad():  
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        batch_preds = probs.argmax(axis=1).cpu().numpy()  # ✅ กลับ CPU เพื่อเก็บ
    
    all_pred_labels.extend(batch_preds)

# ✅ เพิ่มผลลัพธ์เข้า DataFrame
df["predicted_label"] = all_pred_labels
df["predicted_sentiment"] = ["positive" if p == 1 else "negative" for p in all_pred_labels]

# ✅ แปลง label จริงเป็น 0/1
if "sentiment" in df.columns:  # ถ้ามี label จริง
    if df["sentiment"].dtype == object:  
        df["true_label"] = df["sentiment"].apply(lambda x: 1 if x.lower()=="positive" else 0)
    else:
        df["true_label"] = df["sentiment"]

    # ✅ คำนวณ Accuracy
    acc = accuracy_score(df["true_label"], df["predicted_label"])
    print(f"\n Accuracy: {acc*100:.2f}%")

    # ✅ Precision / Recall / F1
    print("\n=== Classification Report ===")
    print(classification_report(df["true_label"], df["predicted_label"], target_names=["negative", "positive"]))
else:
    print("\n No ground-truth labels found, skipping accuracy calculation")

# ✅ แสดงตัวอย่าง 10 row แรก
print("\n=== Sample Predictions (first 10) ===")
for _, row in df.head(10).iterrows():
    print(f"Review: {row['review'][:80]}...")
    if "sentiment" in df.columns:
        print(f"  ✅ True: {row['sentiment']} | 🔮 Predicted: {row['predicted_sentiment']}\n")
    else:
        print(f"  🔮 Predicted: {row['predicted_sentiment']}\n")



✅ Using device: cuda


100%|██████████| 1250/1250 [08:38<00:00,  2.41it/s]


 Accuracy: 96.77%

=== Classification Report ===
              precision    recall  f1-score   support

    negative       0.97      0.97      0.97      5081
    positive       0.97      0.97      0.97      4919

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000


=== Sample Predictions (first 10) ===
Review: I had no background knowledge of this movie before I bought it, but it sounded c...
  ✅ True: Negative | 🔮 Predicted: negative

Review: I am a huge Jane Austen fan and I ordered the movie from Amazon.UK just so I cou...
  ✅ True: Negative | 🔮 Predicted: negative

Review: Nothing to say but Wow! Has anyone actually had somebody sneak up on them in an ...
  ✅ True: Negative | 🔮 Predicted: negative

Review: i like Jane Austin novels. I love Pride and Prejudice and Sense and Sensibility ...
  ✅ True: Negative | 🔮 Predicted: negative

Review: In this day and age of incredibl

In [4]:
# ✅ บันทึกผลลัพธ์
df.to_csv("C:/Users/Lenovo/Desktop/NLP/Final_project/prediction_result/reviews_with_predictions_ModernBERT_Lora_mode_512_b16_ver1_full.csv", index=False)
print("\n✅ Saved predictions to reviews_with_predictions_model >> Done")



✅ Saved predictions to reviews_with_predictions_model >> Done
